In [1]:
import pandas as pd

from GoogleNews import GoogleNews

from urllib.request import urlopen, Request
from bs4 import BeautifulSoup

import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import spacy


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\andre\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [58]:
# My tickers
tickers = ['LTHM', 'USDP', 'SLDP']

# Search Google News
df = pd.DataFrame()
for ticker in tickers:
    googlenews = GoogleNews(lang='en', region='US', period='30d')
    googlenews.search(ticker)
    
    # Convert results to dataframe
    temp = pd.DataFrame(googlenews.result())
    temp = temp[['datetime', 'title', 'desc', 'link']]
    temp.columns = ['Date', 'Headline', 'Desc', 'Link']
    temp.head()

    # Add the ticker
    temp['Ticker'] = ticker

    # Stack the data
    df = pd.concat([df, temp])

# Headline Sentimenet Analysis
vader = SentimentIntensityAnalyzer()
temp = pd.DataFrame(df['Headline'].apply(vader.polarity_scores).tolist())
df['Headline Sentiment'] = temp['compound']

# Description Sentimenet Analysis
vader = SentimentIntensityAnalyzer()
temp = pd.DataFrame(df['Desc'].apply(vader.polarity_scores).tolist())
df['Desc Sentiment'] = temp['compound']

# Get the full article average sentence sentiment
article_scores = []
for link in df['Link']:
    req = Request(url=link,headers={"User-Agent": "Chrome"}) 
    response = urlopen(req) 
    html = BeautifulSoup(response,"html.parser")

    temp_score = []
    for sent in sent_tokenize(html.text.replace('\n', '')):
        temp_score.append(vader.polarity_scores(sent))
    article_scores.append(pd.DataFrame(temp_score)['compound'].mean())

df['Article Sentimenet'] = article_scores
df.head()


,Date,Headline,Desc,Link,Ticker,Headline Sentiment,Desc Sentiment,Article Sentimenet
0,2022-05-14 19:31:34.779559,Can Livent Corporation (LTHM) Add Value To You...,"As of Friday close, Livent Corporation's (NYSE...",https://fostersleader.com/2022/05/14/can-liven...,LTHM,0.34,0.3612,0.097690
1,2022-05-13 22:31:34.811564,Livent Corp (LTHM) Stock Falls -6.60% This Wee...,Livent Corp (LTHM) stock has risen 56.71% over...,https://www.investorsobserver.com/news/stock-u...,LTHM,0.00,0.0000,0.053200
2,2022-05-13 22:31:34.846553,Research Analysts Issue Forecasts for Livent C...,Read Research Analysts Issue Forecasts for Liv...,https://www.defenseworld.net/2022/05/13/resear...,LTHM,0.00,0.1280,0.146197
3,2022-05-13 22:31:34.878909,B. Riley Brokers Raise Earnings Estimates for ...,Livent Co. (NYSE:LTHM – Get Rating) – B. Riley...,https://www.etfdailynews.com/2022/05/13/b-rile...,LTHM,0.00,0.5719,0.213022
4,2022-05-13 22:31:34.926917,Roll up the Sleeves! When the bell rang at pre...,For the readers interested in the stock health...,https://investchronicle.com/2022/05/12/roll-up...,LTHM,0.00,0.6808,0.301505


In [60]:
df.to_csv('data/Google News.csv', index=False)